In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [2]:
daytime = pd.read_csv("DateOfDay01.csv")

#### sheet 3


In [ ]:
####### function ########
def getTreatValue(daytime, t):
    # t is the str.split()
    date_format = "%Y/%m/%d"
    date_format1 = "%Y-%m-%d"
    temp = []    
    for i in range(len(t)):
        try:
            day = str(1911 + int(t[i][:3])) + t[i][3:]
            date_start = datetime.strptime(day, date_format)
            break
        except:
            pass
    temp.append(" ".join(t[1:i-2]))
    
    # add pid and icu time
    temp.append(daytime[0])
    temp.append(datetime.strptime(daytime[1], date_format1))
    
    # 生效時間
    temp.append(date_start)
    i += 2
    
    # 停用時間
    day = str(1911 + int(t[i][:3])) + t[i][3:]
    date_end = datetime.strptime(day, date_format)
    temp.append(date_end)
    return temp

#######  var #######
columns_Cure = {
    'hd' : 'Hemodialysis',
    'Dialysis' : 'Hemodialysis',
    'hemodialysis' : 'Hemodialysis',
    'critical dialysis' : 'Critical dialysis',
    'blood transfusion' : 'blood transfusion',
    'do-not-resuscitate' : 'DO-NOT-RESUSCITATE',
    'dnr' : 'DO-NOT-RESUSCITATE',
    'palliative care family conference fee' : 'Palliative care Family conference Fee'
}
cure_set = ['Hemodialysis', 'Critical dialysiss', 'blood transfusion', 'DO-NOT-RESUSCITATE', 'Palliative care Family conference Fee']
pattern = "|".join(cure_set)


####### clean data ########

date_format = "%Y/%m/%d"
date_format1 = "%Y-%m-%d"
data_set = pd.DataFrame(columns=['治療項目', 'pid', 'icu_day', 'start_day', 'end_day'])
k = 0
for i in daytime.values:
    file_str = "t" + str(i[0]).zfill(4)
    path = file_str + ".xlsx"
    try:
        temp = []
        data = pd.read_excel(path, sheet_name='3.治療')
        if(len(data.columns) == 1):
            
            for value in data.values:
                t = value[0].split()
                try:
#                     print(getTreatValue(i, t))
                    data_set.loc[k] = getTreatValue(i, t)
                    k += 1
                except:
#                     print('f')
                    continue
        else:
            data.columns = data[0:1].values[0]
            data_csv = data[1:]
            data_csv = data_csv.dropna(how='all')
            data_csv = data_csv[[data_csv.columns[0], data_csv.columns[2], data_csv.columns[3]]]
            
            for t in data_csv.values:
                try:
                    temp = []
                    # 治療項目
                    temp.append(" ".join(t[0].split()[1:]))

                    # pid
                    temp.append(i[0])

                    # icu time
                    temp.append(datetime.strptime(i[1], date_format1))

                    # start time
                    date = t[1].split()[0]
                    date = str(1911 + int(date[:3])) + date[3:]
                    date = datetime.strptime(date, date_format)
                    temp.append(date)

                    # end time
                    date = t[2].split()[0]
                    date = str(1911 + int(date[:3])) + date[3:]
                    date = datetime.strptime(date, date_format)
                    temp.append(date)
                    data_set.loc[k] = temp
                    k += 1
                except:
                    continue
        
        print('----------')
        print(path, 'success')
        print('----------')
        
    except:
        print('----------')
        print('no file', path)
        print('----------')
    

    
     
######## main ############

data_set
data_set['icu_day_out'] = data_set['icu_day'] + timedelta(days=27)
cure_day_count = data_set[data_set[data_set.columns[0]].str.lower().str.contains(pattern)]

columns = []
for j in cure_set:
    for i in range(1,29):
        columns.append(j + '_D' + str(i))
cure_days = pd.DataFrame(0, index = daytime.No, columns=columns)

for i in data.values: 
    temp = np.zeros(28, dtype=int)
    day_icu, day_start, day_end, day_icu_out = i[2], i[3], i[4], i[5]
    if(day_start < day_icu):
        day_start = day_icu
    if(day_end > day_icu_out):
        day_end = day_icu_out
        
    dif_start = (day_start - day_icu).days + 1
    dif_end = (day_end - day_icu).days + 1

    for j in cure_set:
        if j in i[0].lower():
            columns = []
            for z in range(dif_start, dif_end+1):
                columns.append(columns_Cure[j] + '_D' + str(z))
            for col in columns:
                cure_days.loc[i[1], col] += 1
                
cure_days.to_csv("./clean_data/sheet3/cure_days.csv")
cure_days.to_csv("./clean_data/sheet3/sheet3.csv")

#### sheet 4

In [ ]:
anti_day_count = pd.DataFrame(columns=['name', 'pid', 'icu_day', 'start_day', 'end_day'])
k = 0
pattern = "|".join(anti_set_lower)
date_format = "%Y-%m-%d"
date_format1 = "%Y/%m/%d"

for i in daytime.values:
    file_str = "t" + str(i[0]).zfill(4)
    path = file_str + ".xlsx"
    try:
        data = read_data(path, file_str)
        target = data[data[data.columns[0]].str.lower().str.contains(pattern)]
        if(target.shape[0] != 0):
            day_icu = datetime.strptime(i[1], date_format)
            day_icu_end = day_icu + timedelta(days=27) # 如果更改成4週，則改為27

            for j in target.values:
                day_start = j[5].split()[0]
                day_start = str(int(day_start[:3]) + 1911) + "/" + day_start[-5:] # 轉換到西元
                day_start = datetime.strptime(day_start, date_format1)

                day_end = j[6].split()[0]
                day_end = str(int(day_end[:3]) + 1911) + "/" + day_end[-5:]
                day_end = datetime.strptime(day_end, date_format1)
                
#                 print('-------------')
#                 print('start:', day_start)
#                 print('end:', day_end)
#                 print('icu_in:', day_icu)
#                 print('-------------')
                
                temp = []
                name = j[0]
                pid = i[0]
                if((day_start >= day_icu) & (day_icu_end >= day_end)):
                    temp = [name, pid, day_icu, day_start, day_end]
                    anti_day_count.loc[k] = temp
                    k += 1
                elif((day_start < day_icu) & (day_icu <= day_end)):
                    # icu前就吃藥，吃到icu期間停止
                    if(day_icu_end >= day_end):
                        temp = [name, pid, day_icu, day_icu, day_end]
                    else:
                        temp = [name, pid, day_icu, day_icu, day_icu_end]
                    anti_day_count.loc[k] = temp
                    k += 1
                elif((day_start < day_icu_end) & (day_icu_end <= day_end)):
                    # icu期間吃藥，並吃到icu六週後
                    if(day_icu <= day_start):
                        temp = [name, pid, day_icu, day_start, day_icu_end]
                    else:
                        temp = [name, pid, day_icu, day_icu, day_icu_end]
                    anti_day_count.loc[k] = temp
                    k += 1
                else:
                    pass
        else:
            pass
    except:
        continue
anti_day_count.to_csv("./clean_data/anti_day_count.csv", index=None)

hyper_day_count = pd.DataFrame(columns=['name', 'pid', 'icu_day', 'start_day', 'end_day'])
k = 0
pattern = "|".join(hyper_set_all)
date_format = "%Y-%m-%d"
date_format1 = "%Y/%m/%d"

for i in daytime.values:
    file_str = "t" + str(i[0]).zfill(4)
    path = file_str + ".xlsx"
    try:
        data = read_data(path, file_str)
        target = data[data[data.columns[0]].str.lower().str.contains(pattern)]
        if(target.shape[0] != 0):
            day_icu = datetime.strptime(i[1], date_format)
            day_icu_end = day_icu + timedelta(days=27)

            for j in target.values:
                day_start = j[5].split()[0]
                day_start = str(int(day_start[:3]) + 1911) + "/" + day_start[-5:] # 轉換到西元
                day_start = datetime.strptime(day_start, date_format1)

                day_end = j[6].split()[0]
                day_end = str(int(day_end[:3]) + 1911) + "/" + day_end[-5:]
                day_end = datetime.strptime(day_end, date_format1)
                
#                 print('-------------')
#                 print('start:', day_start)
#                 print('end:', day_end)
#                 print('icu_in:', day_icu)
#                 print('-------------')
                temp = []
                name = j[0]
                pid = i[0]
                if((day_start >= day_icu) & (day_icu_end >= day_end)):
                    temp = [name, pid, day_icu, day_start, day_end]
                    hyper_day_count.loc[k] = temp
                    k += 1
                elif((day_start < day_icu) & (day_icu <= day_end)):
                    # icu前就吃藥，吃到icu期間停止
                    if(day_icu_end >= day_end):
                        temp = [name, pid, day_icu, day_icu, day_end]
                    else:
                        temp = [name, pid, day_icu, day_icu, day_icu_end]
                    hyper_day_count.loc[k] = temp
                    k += 1
                elif((day_start < day_icu_end) & (day_icu_end <= day_end)):
                    # icu期間吃藥，並吃到icu六週後
                    if(day_icu <= day_start):
                        temp = [name, pid, day_icu, day_start, day_icu_end]
                    else:
                        temp = [name, pid, day_icu, day_icu, day_icu_end]
                    hyper_day_count.loc[k] = temp
                    k += 1
                else:
                    pass
        else:
            pass
    except:
        continue
hyper_day_count.to_csv("clean_data/hyper_day_count.csv", index=None)

#### 抗生素、升壓藥天數

In [113]:
def DF_Datetime_Transform(data):
    date_format = "%Y-%m-%d"
    ICU_Days_Set = []
    Start_Days_Set = []
    End_Days_Set = []
    for i in data.values:
        ICU_Days_Set.append(datetime.strptime(i[2], date_format))
        Start_Days_Set.append(datetime.strptime(i[3], date_format))
        End_Days_Set.append(datetime.strptime(i[4], date_format))
    data['icu_day'] = ICU_Days_Set
    data['start_day'] = Start_Days_Set
    data['end_day'] = End_Days_Set
    return data

######## var #######

anti_set = ['invanz', 'mepem', 'tienam', 'finibax', 'meropem', 'colimycin', 'free-colimycin', 'valcyte', 'cymevene', 'eraxis', 'cancidas', 'mycamine', 'vfend', 'fungizone', 'ambisome', 'ampholipad', 'liposome', 'u-vanco', 'vancomycin', 'vancomycin', 'teicod', 'cubicin', 'targocid', 'zyvox']
columns_Anti = {
    'invanz' : 'Carbapenem',
    'mepem' : 'Carbapenem',
    'tienam' : 'Carbapenem',
    'finibax' : 'Carbapenem',
    'meropem' : 'Carbapenem',
    'colimycin' : 'Colistin',
    'free-colimycin' : 'Colistin',
    'valcyte' : 'Anti_CMV',
    'cymevene' : 'Anti_CMV',
    'eraxis' : 'Anti_fungal',
    'cancidas' : 'Anti_fungal',
    'mycamine' : 'Anti_fungal',
    'vfend' : 'Anti_fungal',
    'fungizone' : 'Anti_fungal',
    'ambisome' : 'Anti_fungal',
    'ampholipad' : 'Anti_fungal',
    'liposome' : 'Anti_fungal',
    'u-vanco' : "Anti_MRSA",
    'vancomycin' : "Anti_MRSA",
    'vancomycin' : "Anti_MRSA",
    'teicod' : "Anti_MRSA",
    'cubicin' : "Anti_MRSA",
    'targocid' : "Anti_MRSA",
    'zyvox' : "Anti_MRSA"
}

hyper_set = ["adrenaline", "levophed", "epirenamin", "epinephrine", "norepinephrine", "pitressin", "dopamin" ,"gipamine"]

######## var #######


####### main anti #######

path = "./clean_data/anti_day_count.csv"
data = pd.read_csv(path)
data = DF_Datetime_Transform(data)

a = ["Carbapenem", "Colistin", "Anti_CMV", "Anti_fungal", "Anti_MRSA"]
columns = []
for j in a:
    for i in range(1,29):
        columns.append(j + '_D' + str(i))
anti_days = pd.DataFrame(0, index = daytime.No, columns=columns)

for i in data.values: 
    temp = np.zeros(28, dtype=int)
    dif_start = (i[3] - i[2]).days + 1
    dif_end = (i[4] - i[2]).days + 1
    
    for j in anti_set:
        if j in i[0].lower():
            columns = []
            for z in range(dif_start, dif_end+1):
                columns.append(columns_Anti[j] + '_D' + str(z))
            for col in columns:
                if anti_days.loc[i[1], col] == 0:
                    anti_days.loc[i[1], col] += 1
    
####### main hyper #######
path = "./clean_data/hyper_day_count.csv"
data = pd.read_csv(path)
data = DF_Datetime_Transform(data)


columns = []
for j in hyper_set:
    for i in range(1,29):
        columns.append(j + '_D' + str(i))
hyper_days = pd.DataFrame(0, index = daytime.No, columns=columns)

for i in data.values: 
    temp = np.zeros(28, dtype=int)
    dif_start = (i[3] - i[2]).days + 1
    dif_end = (i[4] - i[2]).days + 1
    
    for j in hyper_set:
        if j in i[0].lower():
            columns = []
            for z in range(dif_start, dif_end+1):
                columns.append(j + '_D' + str(z))
            for col in columns:
                if hyper_days.loc[i[1], col] == 0:
                    hyper_days.loc[i[1], col] += 1
                    
                    
####### save file #######      

ensure_dir("clean_data/sheet4/anti_days.csv")

anti_days.to_csv("clean_data/sheet4/anti_days.csv")
hyper_days.to_csv("clean_data/sheet4/hyper_days.csv")
sheet4 = hyper_days.join(anti_days)
sheet4.to_csv("clean_data/sheet4/sheet4.csv")